### 0. Import

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1. mon_features.pkl

In [3]:
import pandas as pd
import numpy as np
import pickle

FILE_PATH = '/content/drive/MyDrive/기학/모델 학습/data/mon_features.pkl'
LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER3 = [
    'total_transmission_time', 'std_inter_packet_time', 'avg_outgoing_burst_size',
    'avg_incoming_burst_size', 'num_outgoing_packets', 'incoming_packet_ratio',
    'outgoing_packet_ratio', 'cumul_packets_10pct', 'cumul_packets_30pct',
    'outgoing_order_skew', 'incoming_order_skew', 'cumul_max',
    'num_incoming_first_30', 'outgoing_first_30', 'avg_incoming_order_first_30',
    'avg_outgoing_order_first_30'
]

features_df = pd.read_pickle(FILE_PATH)
X = features_df[FEATURES_VER3]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)

       total_transmission_time  std_inter_packet_time  \
0                        10.14               0.041168   
1                        10.16               0.163930   
2                        11.11               0.066661   
3                        13.36               0.047809   
4                        10.64               0.038760   
...                        ...                    ...   
18995                    43.91               0.143962   
18996                    15.60               0.019465   
18997                    14.93               0.016411   
18998                    19.91               0.033281   
18999                    13.76               0.011074   

       avg_outgoing_burst_size  avg_incoming_burst_size  num_outgoing_packets  \
0                     1.551282                16.666667                 121.0   
1                     1.702128                 9.319149                  80.0   
2                     1.552632                16.315789                 

In [4]:
# 로그 변환
log_cols = []

for col in X.columns:
    # 숫자형이고 & 음수가 하나도 없는 경우만 로그 변환 리스트에 추가
    if pd.api.types.is_numeric_dtype(X[col]) and (X[col] < 0).sum() == 0:
        log_cols.append(col)

# 리스트에 있는 컬럼들 로그 변환
if log_cols:
    X[log_cols] = np.log1p(X[log_cols])

/tmp/ipython-input-1747048184.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[log_cols] = np.log1p(X[log_cols])


### 2. model [K-NN]

#### default 학습

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
# Scaling 추가
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [8]:
y_pred = knn.predict(X_test)
y_pred_proba = knn.predict_proba(X_test)

In [9]:
acc = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")
report = classification_report(y_test, y_pred)

print("=== K-NN 기본 성능 ===")
print("Accuracy:", acc)
print("ROC-AUC:", roc_auc)
print("\nClassification Report:\n", report)

=== K-NN 기본 성능 ===
Accuracy: 0.7281578947368421
ROC-AUC: 0.9386836506159016

Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.60      0.60        40
           1       0.76      0.80      0.78        40
           2       0.76      0.85      0.80        40
           3       0.76      0.88      0.81        40
           4       0.62      0.72      0.67        40
           5       0.60      0.75      0.67        40
           6       0.80      0.93      0.86        40
           7       0.63      0.72      0.67        40
           8       0.61      0.62      0.62        40
           9       0.53      0.70      0.60        40
          10       0.74      0.85      0.79        40
          11       0.82      0.68      0.74        40
          12       0.85      0.85      0.85        40
          13       0.44      0.40      0.42        40
          14       0.53      0.72      0.61        40
          15       0.68      0.75 

#### 하이퍼 파라미터 튜닝 후 학습(GridSearchCV 이용)

In [10]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 15, 21],
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski', 'manhattan', 'euclidean']
}

grid = GridSearchCV(knn, param_grid, cv=5, scoring='f1_macro')
grid.fit(X_train, y_train)

print(grid.best_params_)
print(grid.best_estimator_)

grid_pred = grid.predict(X_test)
grid_pred_proba = grid.predict_proba(X_test)

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance')


In [11]:
acc = accuracy_score(y_test, grid_pred)
roc_auc = roc_auc_score(y_test, grid_pred_proba, multi_class="ovr")
report = classification_report(y_test, grid_pred)

print("=== K-NN 튜닝 후 성능 ===")
print("Accuracy:", acc)
print("ROC-AUC:", roc_auc)
print("\nClassification Report:\n", report)

=== K-NN 튜닝 후 성능 ===
Accuracy: 0.8126315789473684
ROC-AUC: 0.940135393337066

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.68      0.73        40
           1       0.80      0.82      0.81        40
           2       0.89      1.00      0.94        40
           3       0.90      0.93      0.91        40
           4       0.81      0.72      0.76        40
           5       0.81      0.85      0.83        40
           6       0.88      0.90      0.89        40
           7       0.83      0.75      0.79        40
           8       0.70      0.75      0.72        40
           9       0.74      0.78      0.76        40
          10       0.92      0.82      0.87        40
          11       0.94      0.80      0.86        40
          12       0.95      0.88      0.91        40
          13       0.69      0.60      0.64        40
          14       0.74      0.70      0.72        40
          15       0.73      0.82